In [ ]:
!git clone https://github.com/huggingface/diffusers

In [ ]:
%cd diffusers

In [ ]:
!pip install -e .

In [ ]:
!pip install -r examples/dreambooth/requirements_sdxl.txt

In [ ]:
!pip install bitsandbytes

In [ ]:
from accelerate.utils import write_basic_config
write_basic_config()

In [ ]:
!pip install wandb

In [ ]:
import wandb
wandb.login(key = "wandb_api_key") # replace with your wandb key

In [ ]:
!pip install --upgrade peft

In [ ]:
!accelerate launch examples/dreambooth/train_dreambooth_lora_sdxl.py \
    --pretrained_model_name_or_path="stabilityai/stable-diffusion-xl-base-1.0" \
    --instance_data_dir="image_data_directory" \
    --output_dir="/kaggle/working/itay_lora" \
    --mixed_precision="bf16" \
    --instance_prompt="a photo of owhn man" \
    --seed="0" \
    --resolution=1024 \
    --train_batch_size=1 \
    --gradient_accumulation_steps=1 \
    --learning_rate=1e-4 \
    --report_to="wandb" \
    --lr_scheduler="constant" \
    --lr_warmup_steps=0 \
    --num_train_epochs=15 \
    --repeats=10 \
    --checkpointing_steps=750 \
    --output_kohya_format \
    --seed="0" \
    --use_8bit_adam \
    --rank=64 \
    --train_text_encoder \
    --text_encoder_lr=5e-6 \
    --gradient_checkpointing

In [ ]:
%cd /kaggle/working

In [12]:
!rm -rf diffusers

In [13]:
import sys

# Xoá module diffusers khỏi cache
if "diffusers" in sys.modules:
    del sys.modules["diffusers"]

# Loại bỏ tất cả đường dẫn trỏ đến local diffusers
sys.path = [p for p in sys.path if "diffusers" not in p]

In [ ]:
!pip install git+https://github.com/huggingface/diffusers.git --quiet

In [ ]:
from diffusers import DiffusionPipeline
import torch
base_model_id = "stabilityai/stable-diffusion-xl-base-1.0"
pipe = DiffusionPipeline.from_pretrained(base_model_id, torch_dtype=torch.float16)
pipe = pipe.to("cuda")
lora_model_id = "/kaggle/working/itay_lora"
pipe.load_lora_weights(lora_model_id)
prompt = "(wide shot) of analog style modelshoot style 8k close up linkedin profile picture of owhn man, professional jack suite, professional headshots, photo-realistic, 4k, high-resolution image, workplace settings, upper body, modern outfit, professional suit, business, blurred background, glass building, office window. crisp details, neutral expression, high-definition, sharp focus"
negative_prompt = "asian, , , multiple heads, 2 heads, elongated body, double image, 2 faces, multiple people, double head, , (nsfw)+, naked, nude, porn, (deformed iris, deformed pupils, semi-realistic, cgi, 3d, render, sketch, cartoon, drawing, anime:1.4), text, close up"
image = pipe(prompt, negative_prompt = negative_prompt, num_inference_steps=50, guidance_scale = 7.5).images[0]

In [ ]:
image

In [ ]:
import os 
import numpy as np 
output_dir = "/kaggle/working/generated_images"
os.makedirs(output_dir, exist_ok=True)
prompt = "(wide shot) of analog style modelshoot style 8k close up linkedin profile picture of owhn man, professional jack suite, professional headshots, photo-realistic, 4k, high-resolution image, workplace settings, upper body, modern outfit, professional suit, business, blurred background, glass building, office window. crisp details, neutral expression, high-definition, sharp focus"
negative_prompt = "asian, , , multiple heads, 2 heads, elongated body, double image, 2 faces, multiple people, double head, , (nsfw)+, naked, nude, porn, (deformed iris, deformed pupils, semi-realistic, cgi, 3d, render, sketch, cartoon, drawing, anime:1.4), text, close up"
for i in range(10):
    seed = np.random.randint(0, 999999)
    generator = torch.manual_seed(seed)
    image = pipe(prompt, negative_prompt = negative_prompt, num_inference_steps=50, guidance_scale=7.5, generator=generator).images[0]
    image.save(f"{output_dir}/image_{i+1:02d}_seed{seed}.png")